# (IAT 460) Week 2 Lab — Part 1: Randomness and Chaos

## 1. Lab Objectivess

In Week 1, we treated data as material and transformations as tools.
This week, we introduce a new ingredient:
**Uncertainty. Variation. Unpredictability.**

Randomness is not the opposite of control. In computational creativity, randomness is often the source of diversity and richness.

You are not expected to:

- understand all the code

- remember every function

- optimize anything

You are encouraged to:

- observe

- experiment

- discuss

- interpret

This lab containes *non-graded* exercises for you to experiment with the notions we cover.

## *Setup*

In [ ]:
! pip install opensimplex matplotlib pillow numpy soundfile imageio noise tqdm
! mkdir outputs

## 2. Randomness and Noise

### 2.1 Imports and functions

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 

from PIL import Image

This is a simple function to display a histogram based on a numpy array to show the distribution of sampled values

In [ ]:
def plot_distribution(samples):
    count, bins, ignored = plt.hist(samples, 15, density=False)
    plt.hist(bins[:-1], bins)
    plt.show()

This function takes an array of pixels and then expands them to a given size, effectively creating a grid of colored squares

In [ ]:
def assign_colors(samples, grid_size=(50,50,3), image_size=(512,512)):
    img = Image.fromarray(np.uint8(samples.reshape(grid_size)))
    return img.resize(image_size, Image.NEAREST)

In [ ]:
# 50 squares by 50 squares with 3 channels for rgb
grid_size = (50, 50, 3)
n_samples = grid_size[0] * grid_size[1] * grid_size[2]

image_size = (512, 512)

### 2.2 Probability distributions

#### Uniform Distribution
- All values equally likely
- Maximum unpredictability

In [ ]:
samples = np.random.uniform(0, 255, n_samples)
plot_distribution(samples)
img = assign_colors(samples)
display(img)

### Uniform Distribution — Pure Chance
- Values cluster around a mean
- Controlled variation

In [ ]:
samples = np.clip(np.random.normal(loc=128, scale=32, size=n_samples), 0, 255)
plot_distribution(samples)
img = assign_colors(samples)
display(img)

#### Parameters to Explore

- `loc` → brightness bias

- `scale` → contrast / variation

### Exponential Distribution — Rare Extremes
- Many small values, few large ones
- Long tail

In [ ]:
samples = np.clip(np.random.exponential(scale=32, size=n_samples), 0, 255)
plot_distribution(samples)
img = assign_colors(samples)
display(img)

## 2.3 Noise Functions — Coherent Randomness

Non-independant variables, noise depends on position.

#### Simplex / Perline Noise (2D)

In [ ]:
from opensimplex import seed, noise2array

In [ ]:
def simplex_grid(size, scale=4.0, seed_val=0):
    seed(seed_val)
    coords = np.linspace(0, scale, size[0])
    return noise2array(coords, coords)

In [ ]:
# Define the grid of pixels and calculate the noise value at each coordinate
samples = np.zeros((50,50,3))
for c in range(3):
    samples[:,:,c] = simplex_grid((50,50), scale=4, seed_val=np.random.randint(1000))

# Display the result
samples = (samples / 2 + 0.5) * 255
img = assign_colors(samples.flatten())
display(img)

### Layered Noise

By layering noise at multiple scales, we get **complex structure**.

In [ ]:
size = (256, 256)
samples = np.zeros(size)

# We add multiple noise values at different scales
samples += simplex_grid(size, scale=2, seed_val=1) * 1.0
samples += simplex_grid(size, scale=6, seed_val=2) * 0.5
samples += simplex_grid(size, scale=12, seed_val=3) * 0.25


samples = np.clip((samples / 2 + 0.5) * 255, 0, 255)
img = Image.fromarray(np.uint8(samples))
display(img)

## 2.4 Chaos: Deterministic but Unpredictable

#### Logistic Map (simple chaotic system)

In [ ]:
# This function computes the logistic map sequence
def logistic_map(r, x0, n=100):
    x = x0
    values = []
    for _ in range(n):
        x = r * x * (1 - x)
        values.append(x)
    return values

In [ ]:
values = logistic_map(r=3.9, x0=0.2)
plt.plot(values)
plt.title("Chaotic Time Series")
plt.show()

In [ ]:
# We define multiple initial values
init_x0 = [0.2, 0.2000001, 0.199998]
n = 40

values = [
    logistic_map(r=3.9, x0=x0, n=n) for x0 in init_x0
]
# For each initial value sequence, we plot
for val in values:
    plt.plot(val, alpha=0.4)
plt.legend([f"x_0 = {x0}" for x0 in init_x0])
plt.title("Chaotic Time Series")
plt.show()

## 2.5 Sound 

### 2.5.1 Audio Noise as Creative Material

#### *Setup*

In [ ]:
import numpy as np
import soundfile as sf


sr = 44100 # sample rate
duration = 3.0 # seconds
n_samples = int(sr * duration)

#### White Noise

In [ ]:
# Define white noise sequence
white = np.random.normal(0, 1, n_samples)
white /= np.max(np.abs(white))

# Save result
sf.write("outputs/white_noise.wav", white, sr)

## 2.6 Animating Noise

#### *Setup*

In [ ]:
from noise import pnoise2, snoise2, pnoise3

plt.rcParams['figure.figsize'] = (6, 6)

#### Generate Animated Noise

We define the perlin noise functions

In [ ]:
def perlin_noise_2d(width, height, scale=50):
    noise_img = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            noise_img[y, x] = pnoise2(x / scale, y / scale)
    return noise_img


noise_img = perlin_noise_2d(256, 256, scale=60)


plt.imshow(noise_img, cmap='gray')
plt.title("2D Perlin Noise")
plt.axis('off')

In [ ]:
plt.imshow(noise_img, cmap='viridis')
plt.title("Perlin Noise with Color Map")
plt.axis('off')

The layered perlin noise function allows the add noise at multiple frequencies

In [ ]:
def layered_perlin(width, height, scale=100, octaves=4, persistence=0.5, lacunarity=2.0):
    img = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            img[y, x] = pnoise2(
            x / scale,
            y / scale,
            octaves=octaves,
            persistence=persistence,
            lacunarity=lacunarity
            )
    return img


layered = layered_perlin(256, 256)


plt.imshow(layered, cmap='gray')
plt.title("Layered Perlin Noise")
plt.axis('off')

We define the simplex noise function

In [ ]:
def simplex_noise_2d(width, height, scale=50):
    img = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            img[y, x] = snoise2(x / scale, y / scale)
    return img


simplex = simplex_noise_2d(256, 256, scale=60)


plt.imshow(simplex, cmap='gray')
plt.title("2D Simplex Noise")
plt.axis('off')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

# Plotting and comparing perlin noise and simplex noise

axs[0].imshow(noise_img, cmap='gray')
axs[0].set_title('Perlin')
axs[0].axis('off')


axs[1].imshow(simplex, cmap='gray')
axs[1].set_title('Simplex')
axs[1].axis('off')


plt.show()

### Animating noise

In [ ]:
from pathlib import Path
from tqdm import tqdm


width, height = 128, 128
scale = 40
frames = 120
speed = 0.1

# We compute noise images for each frame, while varying one parameter

frame_folder = Path("outputs/frames_noise")

imgs = []
for i in range(frames):
    t = i * speed
    img = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            img[y, x] = snoise2(x / scale, y / scale + t)
    imgs.append(img)

In [ ]:
# Function to retrieve the computed images and create a video

def save_animation(imgs, frame_folder, video_fn = "noise_animation.mp4", fps = 30):

    frame_folder.mkdir(exist_ok=True)

    # save images into the frame folder
    for i, img in tqdm(enumerate(imgs), total=len(imgs)):
        fig = plt.figure(figsize=(6, 6))
        plt.imshow(img, cmap='viridis')


        # turn off axis ticks
        plt.xticks([])
        plt.yticks([])


        # save image
        plt.savefig(frame_folder / f'noise_{i:08d}.png', dpi=150)


        # close figure to avoid memory issues
        plt.close(fig)

    # change working directory to frame folder
    old_wd = Path.cwd()
    %cd $frame_folder


    image_path_format = 'noise_%08d.png'


    # create video
    ! ffmpeg -r $fps -i $image_path_format -y $video_fn


    # return to original directory
    %cd $old_wd

In [ ]:
save_animation(imgs, frame_folder)

In [ ]:
# Defining 3D perlin noise gives us an extra dimension to traverse

def perlin_noise_3d(width, height, z, scale=50):
    img = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            img[y, x] = pnoise3(x / scale, y / scale, z)
    return img


perlin_3d = perlin_noise_3d(256, 256, 0, scale=60)


plt.imshow(perlin_3d, cmap='gray')
plt.title("3D Perlin Noise (2D Plane)")
plt.axis('off')

Here we show the similarity between two close `z` values.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

z_offset = 0.4

perlin_3d_close = perlin_noise_3d(256, 256, z_offset, scale=60)

axs[0].imshow(perlin_3d, cmap='gray')
axs[0].set_title('Perlin (z = 0.0)')
axs[0].axis('off')


axs[1].imshow(perlin_3d_close, cmap='gray')
axs[1].set_title('Perlin (z = 0.1)')
axs[1].axis('off')


plt.show()

By linearly varying our 3rd dimension (`z`), we create a video with interesting results.

In [ ]:
width, height = 128, 128
scale = 40
frames = 120
speed = 0.1


frame_folder = Path("outputs/frames_perlin_3d")

imgs = []
for i in range(frames):
    t = i * speed
    img = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            img[y, x] = pnoise3(x / scale, y / scale, t*0.1)
    imgs.append(img)

save_animation(imgs, frame_folder)

We do the same for the layered perlin noise.

In [ ]:
def layered_perlin_3d(width, height, z, scale=100, octaves=4, persistence=0.5, lacunarity=2.0):
    img = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            img[y, x] = pnoise3(
            x / scale,
            y / scale,
            z,
            octaves=octaves,
            persistence=persistence,
            lacunarity=lacunarity
            )
    return img


In [ ]:
width, height = 128, 128
scale = 40
frames = 240
speed = 0.1

octaves=4 
persistence=0.5
lacunarity=2.0

frame_folder = Path("outputs/frames_perlin_layered_3d")

imgs = []
for i in range(frames):
    t = i * speed
    img = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            img[y, x] = pnoise3(
                x / scale,
                y / scale,
                i * 0.1,
                octaves=octaves,
                persistence=persistence,
                lacunarity=lacunarity
            )
    imgs.append(img)

save_animation(imgs, frame_folder, video_fn="vid.mp4")